# Pedestrians recognition without using Deep Learning

## 1.Dependencies

In [7]:
%matplotlib inline
from matplotlib import pyplot as plt
import cv2
import os
import numpy as np
import glob


data_root = 'img1/'
gt_path = './gt/gt.txt'

_W = 1280
_H = 960
_N = 684 # number of frames


In [30]:
def hog(im_path):
    hog = cv2.HOGDescriptor()
    im = cv2.imread(im_path)
    h = hog.compute(im)
    print(h)

In [31]:
class Test:
    def __init__(self):
        self.dataset_path = data_root
        self.les_im_path = self.get_dataset_im_path(self.dataset_path)
    
    def get_dataset_im_path(self, path):
        """
        Get images path using glob
        :param path: string, dataset folder
        :return les_im_path: list of string, lsit sorted of every .jpg file in the dataset
        """
        les_im_path = glob.glob(path + "*.jpg")
        les_im_path.sort()
        return les_im_path
        
    def test_hog(self):
        test_im = self.les_im_path[0]
        hog(test_im)

tester = Test()
tester.test_hog()

[[0.18076737]
 [0.04871697]
 [0.18816349]
 ...
 [0.2543134 ]
 [0.07930414]
 [0.07784366]]


## Interface function

In [2]:
def pedestrians(data_root, _W, _H, _N):
    ''' Return a list of bounding boxes in the format frame, bb_id, x,y,dx,dy '''
    return [[1,1,617,128,20,50]]

## N.Evaluation

In [ ]:
def format_id(frame):
    assert _N >= frame
    return '{:03d}'.format(frame)


def read_frame(root, frame):
    """Read frames and create integer frame_id-s"""
    assert _N >= frame
    return cv2.imread(os.path.join(root,format_id(frame)+'.jpg'), cv2.IMREAD_UNCHANGED)
    

def read_gt(filename):
    """Read gt and create list of bb-s"""
    assert os.path.exists(filename)
    with open(filename, 'r') as file:
        lines = file.readlines()
    # truncate data (last columns are not needed)
    return [list(map(lambda x: int(x), line.split(',')[:6])) for line in lines]


def annotations_for_frame(solution, frame):
    assert _N >= frame
    return [bb for bb in solution if int(bb[0])==int(frame)]


def evaluate_solution(gt, solution, N):
    """Caclulate evaluation metric"""
    score = []
    #for frame in [300]:
    for frame in range(1, N):
        bbs_sol = annotations_for_frame(solution, frame)
        bbs_gt = annotations_for_frame(gt, frame)
        black_sol = np.zeros((_H, _W))
        black_gt = np.zeros((_H, _W))
        for bb in bbs_sol:
            x, y = bb[2:4]
            dx, dy = bb[4:6]
            cv2.rectangle(black_sol, (x, y), (x+dx, y+dy), (255), -1)
        for bb in bbs_gt:
            x, y = bb[2:4]
            dx, dy = bb[4:6]
            cv2.rectangle(black_gt, (x, y), (x+dx, y+dy), (255), -1)
        # intersection over union
        intersection = black_sol * black_gt
        intersection[intersection > 0.5] = 1
        union = black_sol + black_gt
        union[union > 0.5] = 1
        if not union.any():
            continue
        score.append(intersection.sum()/union.sum())
        
        
    return np.asarray(score).mean()
    

def show_annotation(solution, frame):
    assert _N >= frame
    im = read_frame(data_root, frame)
    bbs = annotations_for_frame(solution, frame)
    for bb in bbs:
        x, y = bb[2:4]
        dx, dy = bb[4:6]
        cv2.rectangle(im, (x, y), (x+dx, y+dy), (0,255,0), 10)
    plt.imshow(im)
    plt.title('Annotations for frame {}.'.format(frame))
    plt.show()
    return

gt = read_gt(gt_path)

show_annotation(gt, 300)

print('A perfect score... {}'.format(evaluate_solution(gt, gt, _N)))

# your solution will be tested simply by changing the dataset
# and changing the module, i.e., the following has to work 
# with simply using your module 
sol = pedestrians(data_root, _W, _H, _N)
print('A great score! {}'.format(evaluate_solution(sol, gt, _N)))

